In [1]:
# import que nous avons besoin:
import copy
import pandas as pd
import numpy as np
import re
import os
import math
import random
import sys
from threading import Thread
import time
import dist_levenstein
import json
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',800)



# fonction general:
## fonction d'insertion dictionnaire
def insert_word(dic,word):
    word=word.lower()
    word=word.replace(".","")
    word=word.replace("!","")
    word=word.replace("\n","")
    word=word.replace("#","")
    word=word.replace("()","")
    word=word.replace(")","")
    word=word.replace(",","")
    word=word.replace("\"","")
    if word not in dic:
        dic[word]=1
    else:
        dic[word]+=1
## fonction de recomendation de mot en cas de faute d'ortographe 
def recommended_word(word):
    # On reduit le champ de recherche ou sinon trop long 
    tmp=datframe_word.loc[(datframe_word["word"].str.len()<=len(word)+1) & (datframe_word["word"].str.len()>=len(word)-1)]
    tmp["dist_levenstein"]=tmp["word"].apply(lambda x : dist_levenstein.DistanceDeLevenshtein(x.lower(),word.lower()))
    return list(tmp.loc[tmp["dist_levenstein"]<3].sort_values("dist_levenstein",ascending=1)["word"])    
## fonction de recomendation de pays en cas de faute d'ortographe
def recommended_country(country):
    tmp=country_for_leven.copy()
    tmp["dist_levenstein"]=tmp["country"].apply(lambda x : dist_levenstein.DistanceDeLevenshtein(x.lower(),country.lower()))
    return list(tmp.loc[tmp["dist_levenstein"]<3].sort_values("dist_levenstein",ascending=1)["country"])   






# Quelques variable globale
tweet = pd.read_csv('./tweets.csv', sep = ',') #Mon dataframe tweet
tweet["word"]=np.nan

liste_date=list(tweet["date"])
date_def=liste_date[int(len(liste_date)/2)]
date_def=date_def[:11]+re.sub(r'[0-9]',"0",date_def[11:]) #Une date aléatoire
                    
                    
#Un dataFrame de tous mes mots contenue dans tweet
dict_word={}
for i in tweet["text"]:
    for word in i.split(" "):
        insert_word(dict_word,word)
datframe_word=pd.DataFrame.from_dict(dict_word,orient="index",columns=["count_word"]).sort_values("count_word",ascending=0)
datframe_word['word']=datframe_word.index
datframe_word['dist_levenstein']=np.nan
datframe_word.index=[i for i in range(0,len(datframe_word))]
#Un dataFrame de tous mes pays contenue dans tweet
country_for_leven=pd.DataFrame((tweet["place_country"].unique()),columns=["country"]).dropna()
datframe_word['dist_levenstein']=np.nan
dict_word={}
word="hahpy"

# Fonction for Javascript pour faire des requetes

In [10]:
def tweet_by_name(name):
    return tweet[tweet["user_name"]==name].to_json()
def tweet_by_place_name(name):
    return tweet[tweet["place_name"]==name].to_json()
def tweet_by_id(name):
    return tweet[tweet["id"]==name].to_json()
def tweet_by_user_id(name):
    return tweet[tweet["user_id"]==name].to_json()
def search_by_word_only(res,word):
    res=res[res["text"].str.contains(r"\b"+word+r"\b",case=False)]
    tmp=tweet[tweet["text"].str.contains(r"\b"+word+r"\b",case=False)]
                                         
    if(len(tmp)==0):
        liste=recommended_word(word)
        #print("Aucun mot vous voulez dire plutot",liste)
    else:
        liste=[]
    if(len(res)==0):
        print("Aucun mot est present pour",word)
    return res,liste
def search_by_word_or(word):
    tmp=tweet[tweet["text"].str.contains(r"\b"+word+r"\b",case=False)]
    tmp["word"]=word
    if(len(tmp)==0):
        liste=recommended_word(word)
        #print("Aucun mot vous voulez dire plutot",liste)
    else:
        liste=[]
    print(word,len(tmp))
    return tmp,liste

def search_by_country(res,country):
    tmp=res[res["place_country"].str.contains(r"\b"+country+r"\b",case=False,na=False)]
    tmp["country"]=country
    verif=len(country_for_leven[country_for_leven["country"].str.contains(r"\b"+country+r"\b",case=False)])
    if(verif==0):
        liste=recommended_country(country)
    else:
        liste=[]
    return tmp,liste
#Cherche tweet après la date de la forme jj/mm/aaaa
def search_by_date1(res,date):
    #J'attribue la meme forme de date grace a date_def
    date_res=date[6:]+"-"+date[3:5]+"-"+date[:2]+'T'+date_def[11:]
    print(date_res)
    return res[res["date"]>date_res]
#Cherche tweet avant la date de la forme jj/mm/aaaa
def search_by_date2(res,date):
    #J'attribue la meme forme de date grace a date_def
    date_res=date[6:]+"-"+date[3:5]+"-"+date[:2]+'T'+date_def[11:]
    print(date_res)
    return res[res["date"]>date_res]

def get_union_hastag(res):
    liste=[]
    #print(len(res))
    for i in range(0,3):
        union_hastag=res.copy()
        union_hastag[res.columns]
        union_hastag=union_hastag.drop(columns=["hashtag_"+str((i+1)%3),"hashtag_"+str((i+2)%3)])
        union_hastag=union_hastag.rename(columns={"hashtag_"+str(i):"hashtag"})
        #print(len(union_hastag))
        #union_hastag=union_hastag.dropna(subset=["hashtag"])
        #print(len(union_hastag))
        liste.append(union_hastag)
    return pd.concat(liste,ignore_index=True)

def get_union_hahstag(res):
    liste=[]
    #print(len(res))
    for i in range(0,3):
        union_hastag=res.copy()
        union_hastag[res.columns]
        union_hastag=union_hastag.drop(columns=["hashtag_"+str((i+1)%3),"hashtag_"+str((i+2)%3)])
        union_hastag=union_hastag.rename(columns={"hashtag_"+str(i):"hashtag"})
        #print(len(union_hastag))
        #union_hastag=union_hastag.dropna(subset=["hashtag"])
        #print(len(union_hastag))
        liste.append(union_hastag)
    return pd.concat(liste,ignore_index=True).reset_index()
def search_by_serveral_word_or(words):
    listes_mot=words.split("_")
    dic_corrected=dict()
    find=False
    res=tweet.copy()
    cpt=0
    for i in range(0,len(listes_mot)):
        word=listes_mot[i]
        if(len(word)==0):
            continue
        tmp,l=search_by_word_or(word)
        if len(l)>0:
            dic_corrected[word]=l
        elif len(tmp)==0:
            True
        else:
            if(cpt>0):
                res=pd.concat([res, tmp], ignore_index=True)
                res.drop_duplicates(subset=["id"])
            else:
                res=tmp
                cpt+=1
            find=True
    for i in dic_corrected:
        print("Aucun mot ne contient",i,"vous voulez dire pour ce mot",dic_corrected[i])
    return res,dic_corrected

def search_by_serveral_country(base,countrys):
    listes_mot=countrys.split("_")
    dic_corrected=dict()
    find=False
    res=base.copy()
    cpt=0
    for i in range(0,len(listes_mot)):
        country=listes_mot[i]
        if(len(country)==0):
            continue
        tmp,l=search_by_country(base,country)
        if len(l)>0:
            dic_corrected[country]=l
        elif len(tmp)==0:
            True
        else:
            if(cpt>0):
                res=pd.concat([res, tmp], ignore_index=True)
                res.drop_duplicates(subset=["id"])
            else:
                res=tmp
                cpt+=1
            find=True
    for i in dic_corrected:
        print("Aucun mot ne contient",i,"vous voulez dire pour ce mot",dic_corrected[i])
    return res,dic_corrected

def request_tweet(words="",name_country="",date_start="",date_end=""):
    #Requetes mot:
    res,l=search_by_serveral_word_or(words)
    words=words.split("_")
    countrys=name_country.split("_")

    #Requetes pays
    reco_country={}
    if name_country!="":
        res,reco_country=search_by_serveral_country(res,name_country)
    #Requetes dates
    if date_start!="":
        res=search_by_date1(res,date_start)
    if date_end!="":
        res=search_by_date2(res,date_end)
    #Avoir le plus grand nombre de tweet par  pays :
    pays=res.groupby("place_country").count().sort_values("id",ascending=0)
    pays=pays[['id']].rename(columns={"id":"total_tweets"}).reset_index()
    pays["count_tweet_per_word"]=np.nan
    for country in pays["place_country"].unique():
        count_tweet_per_word=pd.DataFrame()
        for word in words:
            if(len(word)==0):
                continue
            count_tweet_per_word=count_tweet_per_word.append({"word":word,"count_tweet":len(res.loc[(res["place_country"]==country)&(res["word"]==word)])},ignore_index=True)
            #count_tweet_per_word.append({"word":word,"count_tweet":len(res[res["word"]]==word)},ignore_index=True)
        pays.loc[pays["place_country"]==country,"count_tweet_per_word"]=pays.loc[pays["place_country"]==country,"count_tweet_per_word"].apply(lambda x : count_tweet_per_word.to_dict(orient='records'))

    #Avoir le top 10 hashtag:
    union_hashtag=get_union_hastag(res).reset_index().groupby('hashtag').count().sort_values("index",ascending=0).iloc[0:10][["id"]].reset_index().rename(columns={"id":"count_hashtag"})
    #Avoir les longueur :
    latitude_longitude=pd.DataFrame()
    for word in words:
        how_co=len(res[res["word"]==word])
        if(how_co)==0:
            continue

        latitude_longitude=latitude_longitude.append({"word":word,"coordinations":res[res["word"]==word][["longitude","latitude"]].copy().to_dict(orient='records')},ignore_index=True)
    latitude_longitude_by_country=pd.DataFrame()
    try:
        for country in res["place_country"].unique():
            print(country)
            if(len(country)==0):
                countinue
            lat_lon=np.mean(res[res["place_country"]==country][["longitude","latitude"]])
            how_co=len(res[res["place_country"]==country])
            if(how_co)==0:
                continue
            latitude_longitude_by_country=latitude_longitude_by_country.append({"country":country,"longitude":lat_lon["longitude"],
                                                                                "latitude":lat_lon["latitude"],"count":how_co},ignore_index=True)
    except:
        print("grossee erreur je peux pas faire la map")
        True

    print(l)
    count_tweet=pd.DataFrame()
    count_tweet=count_tweet.append({"word_count":len(words),"totally_tweet_count":len(res),"tweets_per_country":pays.to_dict(orient="records"),"top_hashtags":union_hashtag.to_dict(orient="records"),"tweets_latitude_longitude":latitude_longitude.to_dict(orient="records"),"latitude_longitude_by_country":latitude_longitude_by_country.to_dict(orient="records"),"recommended_word":l,"recommended_country":reco_country},ignore_index=True)
    return count_tweet.to_dict(orient="records")


In [16]:
request_tweet(words="Mau")

France 54
France
França
Francia
Brasil
Bahrain
Italie
日本
Royaume-Uni
Peru
{}


/home/brami/Bureau/Projet_language_dynamique/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[{'latitude_longitude_by_country': [{'country': 'France',
    'latitude': 47.87854065428572,
    'longitude': 2.5551584911904763},
   {'country': 'França', 'latitude': 48.8567, 'longitude': 2.3508},
   {'country': 'Francia', 'latitude': 47.970285242, 'longitude': 2.386582764},
   {'country': 'Brasil', 'latitude': -11.0, 'longitude': -53.0},
   {'country': 'Bahrain', 'latitude': 26.23844302, 'longitude': 50.59505533},
   {'country': 'Italie', 'latitude': 45.09950585, 'longitude': 7.64490582},
   {'country': '日本', 'latitude': 34.67740556, 'longitude': 135.20386934},
   {'country': 'Royaume-Uni', 'latitude': 55.8534, 'longitude': -4.23321},
   {'country': 'Peru', 'latitude': -12.07113351, 'longitude': -77.05148359}],
  'recommended_country': {},
  'recommended_word': {},
  'top_hashtags': [{'hashtag': 'france', 'count_hashtag': 4},
   {'hashtag': 'paris', 'count_hashtag': 2},
   {'hashtag': 'love', 'count_hashtag': 2},
   {'hashtag': 'dgstudio', 'count_hashtag': 2},
   {'hashtag': 'ARGENT

In [17]:
request_tweet(name_country="France")

France
{}


/home/brami/Bureau/Projet_language_dynamique/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[{'latitude_longitude_by_country': [{'country': 'France',
    'latitude': 47.47995707524997,
    'longitude': 2.7193864355}],
  'recommended_country': {},
  'recommended_word': {},
  'top_hashtags': [{'hashtag': 'Marée', 'count_hashtag': 8},
   {'hashtag': 'perdu', 'count_hashtag': 6},
   {'hashtag': 'chien', 'count_hashtag': 5},
   {'hashtag': 'paris', 'count_hashtag': 4},
   {'hashtag': 'chat', 'count_hashtag': 4},
   {'hashtag': 'trouvé', 'count_hashtag': 3},
   {'hashtag': 'T36', 'count_hashtag': 2},
   {'hashtag': 'dgstudio', 'count_hashtag': 2},
   {'hashtag': 'eiffeltower', 'count_hashtag': 2},
   {'hashtag': 'coffee', 'count_hashtag': 2}],
  'totally_tweet_count': 160.0,
  'tweets_latitude_longitude': [],
  'tweets_per_country': [{'place_country': 'France',
    'total_tweets': 160,
    'count_tweet_per_word': []}],
  'word_count': 1.0}]

In [5]:
l,tmp=recommended_country("France")

ValueError: too many values to unpack (expected 2)

In [19]:
tmp[tmp'']

,country,dist_levenstein
0,Indonesia,8
1,Mexico,5
2,Thailand,7
3,日本,6
4,United States,12
...,...,...
308,Srbija,5
309,Malta,5
310,Brésil,5
311,Burkina Faso,10


# Fonctions En attente

In [10]:
def tweet_by_name(name):
    return tweet[tweet["user_name"]==name]
def tweet_by_country(country):
    return tweet[tweet["place_country"]==country]
def tweet_by_place_name(name):
    return tweet[tweet["place_name"]==name]
def tweet_by_id(name):
    return tweet[tweet["id"]==name]
def tweet_by_user_id(name):
    return tweet[tweet["user_id"]==name]
def search_by_word(res,word):
    res=res[res["text"].str.contains(r"\b"+word+r"\b",case=False)]
    tmp=tweet[tweet["text"].str.contains(r"\b"+word+r"\b",case=False)]
                                         
    if(len(tmp)==0):
        liste=recommended_word(word)
        #print("Aucun mot vous voulez dire plutot",liste)
    else:
        liste=[]
    if(len(res)==0):
        print("Aucun mot est present pour",word)
    return res,liste

def search_by_serveral_word(words):
    listes_mot=words.split(",")
    dic_corrected=dict()
    find=False
    res=tweet.copy()
    for i in range(0,len(listes_mot)):
        word=listes_mot[i]
        if (i==0 and find==False):
            tmp,l=search_by_word(res,word)
        else:
            tmp,l=search_by_word(res,word)
        if len(l)>0:
            dic_corrected[word]=l
        elif len(tmp)==0:
            True
        else:
            res=tmp
            find=True
    for i in dic_corrected:
        print("Aucun mot ne contient",i,"vous voulez dire pour ce mot",dic_corrected[i])
    return res,dic_corrected
def search_by_word_or(word):
    tmp=tweet[tweet["text"].str.contains(r"\b"+word+r"\b",case=False)]
    if(len(tmp)==0):
        liste=recommended_word(word)
        #print("Aucun mot vous voulez dire plutot",liste)
    else:
        liste=[]
    print(word,len(tmp))
    return tmp,liste

def search_by_serveral_word_or(words):
    listes_mot=words.split(",")
    dic_corrected=dict()
    find=False
    res=tweet.copy()
    cpt=0
    for i in range(0,len(listes_mot)):
        word=listes_mot[i]
        if (i==0 and find==False):
            tmp,l=search_by_word_or(word)
        else:
            tmp,l=search_by_word_or(word)
        if len(l)>0:
            dic_corrected[word]=l
        elif len(tmp)==0:
            True
        else:
            if(cpt>0):
                res=pd.concat([res, tmp], ignore_index=True)
                res.drop_duplicates(subset=["id"])
            else:
                res=tmp
                cpt+=1
            find=True
    for i in dic_corrected:
        print("Aucun mot ne contient",i,"vous voulez dire pour ce mot",dic_corrected[i])
    return res,dic_corrected



In [72]:
res,dic=search_by_serveral_word_or("happy,yappy,Mau")
res

happy 401
yappy 0
Mau 11
Aucun mot ne contient yappy vous voulez dire pour ce mot ['happy', 'appt', '🐽happy', '🥂happy', '(happy', '🎉happy', 'rappa', 'yatay', 'happŷ', 'yayyy', 'yapış', 'apps', 'yapı', 'puppy', 'yapım', 'apply', '2happy', 'happpy']


/home/brami/Bureau/Projet_language_dynamique-Nathan/ouss/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,user_id,user_name,user_screen_name,user_followers_count,text,place_name,place_country,place_country_code,longitude,latitude,lang,date,timestamp,hashtag_0,hashtag_1,hashtag_2
0,1080002955217129472,132400580,Alfonso Bueno,alfonso_b1,134,"Feliz año nuevo! Happy new year! #2019 @ Nuevo Laredo, Tamaulipas https://t.co/dVMk1Pl7Et","Nuevo Laredo, Tamaulipas",Mexico,MX,-99.507100,27.485600,es,2019-01-01T07:29:04+00:00,1546327744665,NaN,NaN,NaN
1,1080005505337249792,2209026173,aylnsnmz,mcrkz16,74,"Happy new year olur inşallah🙏🏼😅 #konuyubiliyorsunTanrım(amin) (@ Home Sweet Home🏡💚 in Toronto, ON) https://t.co/ruozePd27j https://t.co/eM3rOeQHgF","Toronto, Ontario",Canada,CA,-79.397235,43.759598,en,2019-01-01T07:39:12+00:00,1546328352661,konuyubiliyorsunTanrım,NaN,NaN
2,1080005899618537473,35763405,Alex Amado,AlexAmado2002,547,Wishing everyone a Happy New 2019! @ At Cheonggyecheon Stream https://t.co/8imInI7vaN,"Jongno-gu, Republic of Korea",Republic of Korea,KR,126.978992,37.569142,en,2019-01-01T07:40:46+00:00,1546328446665,NaN,NaN,NaN
3,1080006864295874560,702750804894052353,Ananth Vikram,AnanthVikram85,67,"Wishing All a very Happy and Artistic New Year 2019 ahead!! 🎂🍫🙏😇 \nIf around Bangalore South, kindly make over for a solo recital by my Baby Mahati Ananth @ National College Grounds,… https://t.co/MzdwCGo4ri","Bengaluru South, India",India,IN,77.544640,12.939070,en,2019-01-01T07:44:36+00:00,1546328676662,NaN,NaN,NaN
4,1080007355020890113,16356124,markcoccio,markcoccio,1339,"Happy 2019 Everyone.....man oh man the goals I have this year are HUGE and scary, but stand by for updates and check-ins along the way, it is going to be fun. @ Burbank, California https://t.co/g553Myz1Su","Burbank, CA",United States,US,-118.342900,34.169200,en,2019-01-01T07:46:33+00:00,1546328793660,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,1080878759417430016,92357502,Afra Augesti,avraaugesty,691,"Malam ini ceritanya lagi mau nyoba ngedit video di laptop. Tetiba nemu 3 rekaman ""dibuang sayang"" ini.\n.\nEto-etonya, di video itu, kami lagi diwawancara penyiar Capital FM, salah satu… https://t.co/5FvQJGT8iA","Dhaka, Bangladesh",Bangladesh,BD,90.433209,23.813322,in,2019-01-03T17:29:12+00:00,1546536552658,NaN,NaN,NaN
408,1083179888805380097,773709250182934532,jogjajourney,jogjajourney1,33,Mantai yuuk...\n\n📷 by laila_journey \n\nMau?\nReservasi sekarang juga !\nCall/text/WA/TG : +6287705576633\n\nMore information :\naristiana_jogjajourney lopinda_journey @utamipipit nisrinanazih… https://t.co/jT6HS63Yi7,"Danurejan, Indonesia",Indonesia,ID,110.368000,-7.795850,in,2019-01-10T01:53:04+00:00,1547085184657,NaN,NaN,NaN
409,1083401499076640773,2726285827,Cinere Scooter Club,VespaCinere,1782,#Repost from scooter_beang_kerok with https://t.co/bdeEaPnh03 \n ... \n\nReposted from pesi_official - Buat sedulur se vespa mari budayakan kembali 1 VESPA BERJUTA SODARA.. jadi...bagi sedulur yg mau… https://t.co/U2ozy7mmrV,"Lima, Indonesia",Indonesia,ID,106.782747,-6.334487,in,2019-01-10T16:33:40+00:00,1547138020663,Repost,NaN,NaN
410,1085739328632995842,70320798,yono kojiro,yo_kojiro,371,"Masih berpikir mau beli mobil selain Toyota Avanza.....???\n\nHaaaiii Kalimantan Barat... Mau service tanpa antri, yang lagi mager atau yg sambil bersihin rumah, kuras bak mandi, ngepel… https://t.co/rFg6KCK2uc","West Borneo, Indonesia",Indonesia,ID,109.368618,-0.085911,in,2019-01-17T03:23:22+00:00,1547695402664,NaN,NaN,NaN


In [129]:
#Code qui permet de sortir la liste country
l=sorted(list(tweet["place_country"].dropna().unique()))
f=open('liste_country.txt','w')
for ele in l:
    f.write(ele+'\n')


{'yappy': ['happy',
  'appt',
  '🐽happy',
  '🥂happy',
  '(happy',
  '🎉happy',
  'rappa',
  'yatay',
  'happŷ',
  'yayyy',
  'yapış',
  'apps',
  'yapı',
  'puppy',
  'yapım',
  'apply',
  '2happy',
  'happpy']}

In [93]:
search_by_word("happy")

,id,user_id,user_name,user_screen_name,user_followers_count,text,place_name,place_country,place_country_code,longitude,latitude,lang,date,timestamp,hashtag_0,hashtag_1,hashtag_2
16,1080004972647989249,99873362,JON RECRDS,juan_ON_,523,18-19\n2018 fue el año de la fuerza de atracción. De comprobar que lo que deseas te llega... que vengan muchos años así o mejores!🙌🏽 #gracias! \n#photography #2018 \n#2019 #happynewyear… https://t.co/tdrsJQIhhe,"Monterrey, Nuevo León",México,MX,-100.300000,25.666700,es,2019-01-01T07:37:05+00:00,1546328225658,NaN,NaN,NaN
38,1080007313090625536,799284810,Oscar Ornelas,oscarornelas512,37,#happynewyear @ The Skylark Lounge https://t.co/X6XKcUGfEz,"Austin, TX",United States,US,-97.704650,30.284490,en,2019-01-01T07:46:23+00:00,1546328783663,happynewyear,NaN,NaN
58,1080009531869020161,235531227,Khyati Ratan,KhyatiRatan,109,"Celebrate endings – for they precede new beginnings - Jonathan Lockwood Huie, author\n#newyearseve #nye #newyear #christmas #party #newyears #happynewyear #dj #love #music #holiday… https://t.co/9iI7vp5p2p","Mumbai, India",India,IN,72.876400,19.062100,en,2019-01-01T07:55:12+00:00,1546329312661,newyearseve,nye,newyear
67,1080010236512034817,233411963,Anthony BOY Flores,boyrack87,814,#happynewyear 🎇 everyone. #2019 and beyond is about growth. I've kept it real for long enough. There's not one dam person on this earth that can make me feel less confident other then… https://t.co/tzwwzPGIol,"Los Angeles, CA",United States,US,-118.243000,34.052200,en,2019-01-01T07:58:00+00:00,1546329480661,happynewyear,NaN,NaN
72,1080010769171804160,20509373,The Real JWT,RealJWT,121,Champagne Wishes &amp; Caviar Dreams! 🥂 Happy 2019! #champagne #newyear #2019 #cheers #california #losangeles #family #newbeginnings #happy #celebrate #crystal #beautiful #countdown #holiday… https://t.co/MvSlLpeeOq,"Los Angeles, CA",United States,US,-118.243000,34.052200,en,2019-01-01T08:00:07+00:00,1546329607657,champagne,newyear,cheers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18572,1085701441489113088,2573230169,Jean Braga,Everson_braga31,16,"Felicidade sem motivo, melhor impossível!\n.\n.\n.\n.\n.\n.\n.\n😄 #happy #toptags #happydays #happyday #smile #fun #instahappy #goodmood #sohappy #happier #excited #feelgood #smiling #funtimes… https://t.co/WxkbWUAZX7","Fortaleza, Brasil",Brasil,BR,-38.540554,-3.714208,pt,2019-01-17T00:52:49+00:00,1547686369665,happy,toptags,happydays
18709,1085718201915170816,46345790,WedKes,WedKes,35,NY Party 2019 #newyear #party #data #architecture #team #gsb #dinner #happy #perfect @ ครัวกรุงเทพ พญาไท https://t.co/I3RRgyTOef,"Thanon Phaya Thai, Thailand",Thailand,TH,100.532832,13.755291,th,2019-01-17T01:59:25+00:00,1547690365662,newyear,party,data
18772,1085728998062055424,183895938,Harsh Gupta,fairngood,66,Good Morning Everyone \n\n#behappy #stayhappy #keepsmiling #keepgrowing @ Madhav Niketan https://t.co/Fr75xNwDZP,"Ludhiana, India",India,IN,75.777905,30.870134,en,2019-01-17T02:42:19+00:00,1547692939664,behappy,stayhappy,keepsmiling
18807,1085733582428037120,65730236,Bullet Mckenzie,BulletMcK,592,My favorite part of Rockn Roll Weekend was being able to cook and feed the fam at @eatbeerfish. Thank you all for coming down. So happy to hang with my rockn roll sister and… https://t.co/WFT9qIDwa9,"San Diego, CA",United States,US,-117.130630,32.762640,en,2019-01-17T03:00:32+00:00,1547694032662,NaN,NaN,NaN


# Brouillon

In [ ]:
def obs(x):
    if dist_levenstein.DistanceDeLevenshtein(x.lower(),"yahpy".lower())<3:
        print(x,dist_levenstein.DistanceDeLevenshtein(x.lower(),"yappy".lower()))
datframe_word["word"].apply(obs)
tmp=tweet[tweet["text"].str.contains("(?i)\b"+word+"\b"]


#### Bon format pour un mot

In [ ]:
#Requetes mot:
res,l=search_by_word_only(tweet,"Mau")
res.groupby("place_country").count()
#Avoir le plus grand nombre de tweet par  pays :
res,l=search_by_word_only(tweet,"Mau")
res.groupby("hashtag_0").count().sort_values("id",ascending=0)
#Compte de tweet par mot
words="Mau,Salat"
count_tweet_per_word=pd.DataFrame()
res,l=search_by_serveral_word_or(words)
for word in words.split(","):
    count_tweet_per_word=count_tweet_per_word.append({"word":word,"count_tweet":len(res[res["word"]==word])},ignore_index=True)
    #count_tweet_per_word.append({"word":word,"count_tweet":len(res[res["word"]]==word)},ignore_index=True)

#Avoir le top 10 hashtag:
union_hashtag=get_union_hastag(tweet).groupby('hashtag').count().sort_values("index",ascending=0).iloc[0:10]
union_hashtag